# CPLEX Solver WIKI

Provide an array containing dictionaries that define your variables an constraints

## Variables defintion

This example show you how to define a decision variable:

```
Xs = {
    "name" : [X(i) for i in I],
    "coef" : [c[i] for i],
    "type" : ["C" if relaxation else "I" for i in I],
    "ub"   : [1 for i in I],
    "lb"   : [0 for i in I],
}
```

You can also use the compact syntax:

```
Xs = {
    "loop" : "for i in I"
    "name" : "X(i)",
    "coef" : "c[i]",
    "type" : ""C" if relaxation else "I"",
    "ub"   : "1",
    "lb"   : "0",
}
```

by adding the attribute loop the function will recreate the list by evaluating the composed of the attribute and the loop expression.

# Source code

In [40]:
%%file "../pkg/helpers/cplex_solve_extract_variables.py"

def cplex_solve_extract_variables(variables):
    obj = [];ub = [];lb = [];colnames = [];types = [];
    for v in variables:
        if "loop" in v:
            ## evaluation of the loop
            v["coef"] = eval("[" + v["coef"] + v["loop"] + "]")
            v["ub"]   = eval("[" + v["ub"] + v["loop"] + "]")
            v["lb"]   = eval("[" + v["lb"] + v["loop"] + "]")
            v["name"] = eval("[" + v["name"] + v["name"] + "]")
            v["type"] = eval("[" + v["type"] + v["loop"] + "]")
            
        # Concatenation of variables attributes
        obj      = obj + v["coef"]
        ub       = ub  + v["ub"]
        lb       = lb  + v["lb"]
        colnames = colnames + v["name"]
        types    = types + v["type"]
        
    return obj, ub, lb, colnames, types

Overwriting ../pkg/helpers/cplex_solve_extract_variables.py


In [41]:
%%file "../pkg/helpers/cplex_solve_extract_constraints.py"

def cplex_solve_extract_constraints(constraints):
    rows = []; senses = []; rhs = [];
    for c in constraints:
        if "loop" in c:
            c["lin_expr"] = eval("[" + c["lin_expr"] + c["loop"] + "]")
            c["senses"]   = eval("[" + c["senses"] + c["loop"] + "]")
            c["rhs"]      = eval("[" + c["rhs"] + c["loop"] + "]")
            
        # Concatenation of variables attributes    
        rows   = rows   + c["lin_expr"]
        senses = senses + c["senses"]
        rhs    = rhs    + c["rhs"]
        
    return rows, senses, rhs

Overwriting ../pkg/helpers/cplex_solve_extract_constraints.py


In [42]:
%%file "../pkg/helpers/cplex_solve.py"

import cplex
import pkg.helpers.cplex_solve_extract_constraints

def cplex_solve_extract_constraints(constraints):
    rows = []; senses = []; rhs = [];
    for c in constraints:
        if "loop" in c:
            c["lin_expr"] = eval("[" + c["lin_expr"] + c["loop"] + "]")
            c["senses"]   = eval("[" + c["senses"] + c["loop"] + "]")
            c["rhs"]      = eval("[" + c["rhs"] + c["loop"] + "]")
            
        # Concatenation of variables attributes    
        rows   = rows   + c["lin_expr"]
        senses = senses + c["senses"]
        rhs    = rhs    + c["rhs"]
        
    return rows, senses, rhs

def cplex_solve(variables,constraints, minimize=True, path=None, verbose=False):
    
    #####################################################################
    # Extract variables
    obj, ub, lb, colnames, types = cplex_solve_extract_variables(variables)
    #####################################################################
    # Extract constraints
    rows, senses, rhs = cplex_solve_extract_constraints(constraints)
    #####################################################################
    # Creating problem
    prob = cplex.Cplex()
    
    #Disable logging
    if not verbose:
        prob.set_log_stream(None)
        prob.set_error_stream(None)
        prob.set_warning_stream(None)
        prob.set_results_stream(None)
    
    ## Objective function sense
    if minimize:
        prob.objective.set_sense(prob.objective.sense.minimize)
    else:
        prob.objective.set_sense(prob.objective.sense.maximize)
    ## Objective function
    prob.variables.add(obj=obj,ub=ub,lb=lb,names=colnames,types=types)
    ## Constraintes
    prob.linear_constraints.add(lin_expr=rows,senses=senses, rhs=rhs)

    #####################################################################
    # Saving the linear problem formulation into a file
    if path:
        prob.write(path) # print the formulation into a file

    #####################################################################
    # Solving problem
    prob.solve()
    
    return prob

Overwriting ../pkg/helpers/cplex_solve.py


# Test

In [43]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [48]:
import pkg.helpers.cplex_solve_extract_variables as cpx

In [45]:
I = range(5); J = range(5);
c = [[1 for i in I] for j in J]

Xs = {
    "name" : ["X_" + str(i) + "_" + str(j) for i in I for j in J],
    "coef" : [c[i][j] for i in I for j in J],
    "type" : ["I" for i in I for j in J],
    "ub"   : [1 for i in I for j in J],
    "lb"   : [0 for i in I for j in J],
}


variables = [Xs]

In [49]:
cpx.cplex_solve_extract_variables(variables)

([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 ['X_0_0',
  'X_0_1',
  'X_0_2',
  'X_0_3',
  'X_0_4',
  'X_1_0',
  'X_1_1',
  'X_1_2',
  'X_1_3',
  'X_1_4',
  'X_2_0',
  'X_2_1',
  'X_2_2',
  'X_2_3',
  'X_2_4',
  'X_3_0',
  'X_3_1',
  'X_3_2',
  'X_3_3',
  'X_3_4',
  'X_4_0',
  'X_4_1',
  'X_4_2',
  'X_4_3',
  'X_4_4'],
 ['I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I'])